# Writing and retrieving co-simulation setups

This notebook shows how to write and retrieve co-simulation setups from a database.

**Note**: In order to have the 3DCityDB populated with the data required for this example, run notebook [DBLayerBasics](./DBLayerBasics.ipynb) exactly once.

## Example: A simple co-simulation setup

For the following, consider that a co-simulation setup called `sim` has been defined with the help of the  [IntegrCiTy co-simulation deployment API](https://github.com/IntegrCiTy/ictdeploy):

In [1]:
from ictdeploy import Simulator
import os.path

# Create simulation setup.
sim = Simulator()

# Add meta model.
sim.edit.add_meta(
  name = 'BaseMeta',
  set_attrs = [ 'a' ],
  get_attrs = [ 'b' ]
  )

# Add model based on meta model.
sim.edit.add_model(
  name = 'BaseModel',
  meta = 'BaseMeta',
  image = 'integrcity/ict-simple',
  wrapper = os.path.join( 'tests', 'wrappers', 'base_wrap.py' ),
  command = None,
  files = [ os.path.join( 'tests', 'files_to_add', 'empty_file_for_testing_purpose.txt' ) ]
  )

# Add node based on model.
sim.edit.add_node(
  name = 'Base0',
  model = 'BaseModel',
  init_values = { 'c': 0.5 },
  is_first = True
  )

# Add another node based on model.
sim.edit.add_node(
   name = 'Base1',
   model = 'BaseModel',
   init_values = { 'c': 0.25 }
  )

# Define links between nodes.
sim.edit.add_link( get_node = 'Base0', get_attr = 'b', set_node = 'Base1', set_attr = 'a' )
sim.edit.add_link( get_node = 'Base1', get_attr = 'b', set_node = 'Base0', set_attr = 'a' )

# Define simulation groups and sequence.
grp0 = sim.create_group( 'Base0' )
grp1 = sim.create_group( 'Base1' )
sim.create_sequence( grp0, grp1 )

# Define simulation time steps.
sim.create_steps( [60] * 10 )

## Storing a co-simulation setup to the database

Package DBLayer implements the mapping of the [Simulation Package](https://github.com/gioagu/3dcitydb_simulation_pkg) scheme to IntegrCiTy’s concepts for representing such a co-simulation setup, providing basically a persistence layer for co-simulation setups.
For storing co-simulation setups to the extended 3DCityDB, package DBLayer provides class `DBWriter`.
Upon connecting to the database, a co-simulation setup can be assigned a name and written to the database with the help of a single command.
Setting parameters `write_meta_models` and `write_models` to `False` indicates that a co-simulation setup already defining meta-model `MetaBase` and model `BaseModel` have been previously written to the same database (otherwise the corresponding parameters should be omitted or set to `True`).

In [2]:
from dblayer import *

# Define connection parameters.
connect = PostgreSQLConnectionInfo(
    user = 'postgres',
    pwd = 'postgres',
    host = 'localhost',
    port = '5432',
    dbname = 'citydb'
    )

# Create writer and connect to database.
writer = DBWriter( connect )

# Write co-simulation setup to database.
writer.write_to_db(
    sim,
    'TestSim1',
    write_meta_models = True,
    write_models = True
    )

## Retrieving co-simulation setups

For reading co-simulation setups from the extended 3DCityDB, package DBLayer provides class `DBReader`.
Upon connecting to the database, a co-simulation setup stored in the database can be referred to by name and retrieved with the help of a single command.
In the example below, which simply reads back the co-simulation setup written in the previous example, the resulting object `sim_read` would be identical to object `sim` from above.

In [3]:
# Create reader and connect to database.
reader = DBReader( connect )

# Read co-simulation setup from database.
sim_read = reader.read_from_db( 'TestSim1' )

## Associate co-simulation setups with 3DCityDB data

In the current IntegrCiTy toolchain, the association of co-simulation setups with 3DCityDB data happens first and foremost by using the available information to parametrize simulation models.
In the context of urban energy systems simulation, this comprises not only scalar model parameters (e.g., U-values for walls) but also time-series data as provided via the Energy ADE (e.g., electrical load profiles).

The rather trivial approach for such an association is to directly retrieve certain values (according to the example above) and setting them as initial values when defining the co-simulation setup.

However, package DBLayer and the Simulation Package also allow to persistently store associations with attributes of 3DCityDB objects and generic parameters.
This is demonstrated in the following pseudo code snippet, which basically extends the co-simulation setup shown above by defining an instance of class `AssociateCityDBObject` and assigning it to parameter `c` of node `Base0`.

In [4]:
# Create access point.
access = DBAccess()

# Connect to database.
access.connect_to_citydb( connect )

# Query the database using the conditions defined above.
heatpumps = access.get_citydb_objects( 'HeatPump' )
heatpump_id = heatpumps[0].id

# Define table and attribute (table column) name for association.
table = 'citydb_view.nrg8_conv_system_heat_pump'
attribute = 'nom_effcy'

# Define association with the help of the object ID (table row).
associate_object = AssociateCityDBObject(
    table_name = table,
    object_id = heatpump_id,
    column_name = attribute
    )

# Link attribute "c" of "Base0" with the association.
sim.edit.nodes.loc[ 'Base0' ].init_values[ 'c' ] = associate_object

# Write the co-simulation setup to the database (storing the association).
writer.write_to_db(
    sim,
    'TestSim2',
    write_meta_models = False,
    write_models = False
    )

Done.